In [1]:
from python_speech_features import mfcc
import scipy.io.wavfile as wav
import numpy as np
from tempfile import TemporaryFile
import os
import pickle
import random 
import operator
import math
import numpy as np

### 2. Define a function to get the distance between feature vectors and find neighbors:

In [2]:
def getNeighbors(trainingSet, instance, k):
    distances = []
    for x in range (len(trainingSet)):
        dist = distance(trainingSet[x], instance, k )+ distance(instance, trainingSet[x], k)
        distances.append((trainingSet[x][2], dist))
    distances.sort(key=operator.itemgetter(1))
    neighbors = []
    for x in range(k):
        neighbors.append(distances[x][0])
    return neighbors

### 3. Identify the nearest neighbors:

In [3]:
def nearestClass(neighbors):
    classVote = {}
    for x in range(len(neighbors)):
        response = neighbors[x]
        if response in classVote:
            classVote[response]+=1 
        else:
            classVote[response]=1
    sorter = sorted(classVote.items(), key = operator.itemgetter(1), reverse=True)
    return sorter[0][0]

###  4. Define a function for model evaluation:

In [4]:
def getAccuracy(testSet, predictions):
    correct = 0 
    for x in range (len(testSet)):
        if testSet[x][-1]==predictions[x]:
            correct+=1
    return 1.0*correct/len(testSet)

In [5]:
def distance(instance1 , instance2 , k ):
    distance =0 
    mm1 = instance1[0] 
    cm1 = instance1[1]
    mm2 = instance2[0]
    cm2 = instance2[1]
    distance = np.trace(np.dot(np.linalg.inv(cm2), cm1)) 
    distance+=(np.dot(np.dot((mm2-mm1).transpose() , np.linalg.inv(cm2)) , mm2-mm1 )) 
    distance+= np.log(np.linalg.det(cm2)) - np.log(np.linalg.det(cm1))
    distance-= k
    return distance

### 5. Extract features from the dataset and dump these features into a binary .dat file “my.dat”:

In [6]:
#directory = "C:/Users/Adrian/Desktop/CATO_2021/Sistemas_Inteligentes/Repos/Practice_8/Practica_8/genres/"
directory = os.getcwd()+'/genres/'
f= open("my.dat" ,'wb')

for folder in os.listdir(directory):
    if ".mf" not in folder: 
        print ('*********')
        print (folder)
        for file in os.listdir(directory+"/"+folder):  
            (rate,sig) = wav.read(directory+folder+"/"+file)
            mfcc_feat = mfcc(sig,rate ,winlen=0.020, appendEnergy = False)
            covariance = np.cov(np.matrix.transpose(mfcc_feat))
            mean_matrix = mfcc_feat.mean(0)
            feature = (mean_matrix , covariance , i)
            pickle.dump(feature , f)
            
f.close()

1
2
3
*********
blues
4
5
*********
classical
6
7
*********
country
8
9
*********
disco
10
11
*********
hiphop
12
13
14
*********
jazz
15
16
*********
metal
17
18
*********
pop
19
20
*********
reggae
21
22
*********
rock


### 6. Train and test split on the dataset:

In [7]:
dataset = []
def loadDataset(filename , split , trSet , teSet):
    with open("my.dat" , 'rb') as f:
        while True:
            try:
                dataset.append(pickle.load(f))
            except EOFError:
                f.close()
                break  
    for x in range(len(dataset)):
        if random.random() <split :      
            trSet.append(dataset[x])
        else:
            teSet.append(dataset[x])  
trainingSet = []
testSet = []
loadDataset("my.dat" , 0.66, trainingSet, testSet)

### 7. Make prediction using KNN and get the accuracy on test data:

In [8]:
leng = len(testSet)
predictions = []
for x in range (leng):
    predictions.append(nearestClass(getNeighbors(trainingSet ,testSet[x] , 5))) 
accuracy1 = getAccuracy(testSet , predictions)
print(accuracy1)

0.693939393939394
